In [21]:
!pip install scipy
!pip install torch torchvision
!pip install wandb

python3.11(19264) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


python3.11(19266) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


python3.11(19269) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 39.0 MB/s eta 0:00:00 MB/s eta 0:00:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.1 MB/s eta 0:00:00


In [30]:
import torch
import wandb
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

python3.11(19645) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.11(19646) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [17]:
import torch.nn as nn
from torchvision import models
import torch.optim as optim

In [14]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),       # Resize images to 224x224
    transforms.ToTensor(),               # Convert images to PyTorch tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])

In [15]:

# Load Flowers102 dataset
train_data = datasets.Flowers102(
    root="./data",  # Directory to store data
    split="train",  # Options: 'train', 'val', 'test'
    transform=transform,
    download=True   # Download if not already downloaded
)

val_data = datasets.Flowers102(
    root="./data",
    split="val",
    transform=transform,
    download=True
)

test_data = datasets.Flowers102(
    root="./data",
    split="test",
    transform=transform,
    download=True
)

# # Data loaders
# train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
# test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# # Check dataset size
# print(f"Train size: {len(train_data)}")
# print(f"Validation size: {len(val_data)}")
# print(f"Test size: {len(test_data)}")
# Combine all splits
all_data = train_data + val_data + test_data

# Desired split proportions
train_size = int(0.7 * len(all_data))  # 70% for training
val_size = int(0.2 * len(all_data))    # 20% for validation
test_size = len(all_data) - train_size - val_size  # Remaining for test

# Split dataset
train_data, val_data, test_data = random_split(
    all_data, [train_size, val_size, test_size]
)

# Data loaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Check sizes
print(f"Train size: {len(train_data)}")
print(f"Validation size: {len(val_data)}")
print(f"Test size: {len(test_data)}")

Train size: 5732
Validation size: 1637
Test size: 820


In [27]:
# Load pre-trained model
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)


# Save the model
torch.save(model.state_dict(), "flower_model.pth")


mps


TypeError: empty(): argument 'size' failed to unpack the object at pos 2 with error "type must be tuple of ints,but got generator"

In [25]:
model.fc.in_features

2048

In [28]:
# Set device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

# Load pretrained MobileNetV2 model
model = models.mobilenet_v2(pretrained=True)

# Modify the classifier
num_features = model.last_channel
model.classifier = nn.Sequential(
    nn.Linear(num_features, 102)  # 102 classes for Flowers102
)

model = model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

mps
Epoch 1/50, Loss: 2.5337496783998277
Epoch 2/50, Loss: 0.6959839665227466
Epoch 3/50, Loss: 0.2548768334504631
Epoch 4/50, Loss: 0.12292758824510706
Epoch 5/50, Loss: 0.06659283001596729


KeyboardInterrupt: 

In [31]:
lr = 0.0001
epochs = 50
classes = 102
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="flowers",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "MobileNet",
    "dataset": "oxford_102",
    "epochs": epochs,
    }
)

python3.11(19651) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.11(19652) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
python3.11(19653) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /Users/ash/.netrc
python3.11(19659) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.11(19661) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [32]:
# Load pretrained MobileNetV2 model
model = models.mobilenet_v2(pretrained=True)

# Modify the classifier
num_features = model.last_channel
model.classifier = nn.Sequential(
    nn.Linear(num_features, classes)  # 102 classes for Flowers102
)

model = model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(epochs):
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")
    wandb.log({"epoch":epoch+1, "loss": running_loss/len(train_loader)})

torch.save(model.state_dict(), "flower_model_mobilenet.pth")


/Users/ash/miniforge3/envs/py311_env/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ash/miniforge3/envs/py311_env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/50, Loss: 2.512230842643314
Epoch 2/50, Loss: 0.6996534797052543
Epoch 3/50, Loss: 0.2708473992844423
Epoch 4/50, Loss: 0.11584756976614395
Epoch 5/50, Loss: 0.06079761580460601
Epoch 6/50, Loss: 0.03603095568509566
Epoch 7/50, Loss: 0.027243855812897285
Epoch 8/50, Loss: 0.022384466148085064
Epoch 9/50, Loss: 0.03460638436178366
Epoch 10/50, Loss: 0.019363736803643405
Epoch 11/50, Loss: 0.013158965258238215
Epoch 12/50, Loss: 0.020055048601029232
Epoch 13/50, Loss: 0.009340683916687138
Epoch 14/50, Loss: 0.0066460835236486875
Epoch 15/50, Loss: 0.01123515197282864
Epoch 16/50, Loss: 0.03859391737755181
Epoch 17/50, Loss: 0.011177202398216145
Epoch 18/50, Loss: 0.011246235195236902
Epoch 19/50, Loss: 0.015035632026976803
Epoch 20/50, Loss: 0.028260997224909563
Epoch 21/50, Loss: 0.024009958421811462
Epoch 22/50, Loss: 0.01594431357322416
Epoch 23/50, Loss: 0.022044818262414386
Epoch 24/50, Loss: 0.007398745278866651
Epoch 25/50, Loss: 0.013779378923289995
Epoch 26/50, Loss: 0.0

In [33]:
# End the W&B run
wandb.finish()

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50
loss,0.01181


In [34]:
wandb.init(
    # set the wandb project where this run will be logged
    project="flowers",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "ResNet",
    "dataset": "oxford_102",
    "epochs": epochs,
    }
)

wandb: Currently logged in as: ashwanirathee (ashwanirathee-uc-santa-cruz-banana-slugs). Use `wandb login --relogin` to force relogin
python3.11(21178) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [ ]:
model = models.resnet50(pretrained=True)

# Modify the classifier
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 102)  # 102 classes
model = model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")
    wandb.log({"epoch":epoch+1, "loss": running_loss/len(train_loader)})

torch.save(model.state_dict(), "flower_resnet_model.pth")

/Users/ash/miniforge3/envs/py311_env/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ash/miniforge3/envs/py311_env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/50, Loss: 2.3572369952996572


In [ ]:
# End the W&B run
wandb.finish()

In [ ]:
# Save the model
torch.save(model.state_dict(), "flower_model.pth")